# Translating power system data into QCQP

1. `GEN_EST` = `CONV_GEN` + `WIND_FCST`

2. `OBJ`: $\min\lVert $ `DEV` $\rVert$ $$\min Q(x)$$

3. Power balance: `ADMTNCE`$\times$ `THETA` - $I\times$ `DEV` = `GEN_EST` - `DEMAND` $$Ax=b$$

4. Temp. constraint: $\lVert$ `THETA_i` - `THETA_k` $\rVert$ = $\Delta T_{lim}$

## Toy example

Two renewable nodes, three total buses, two time steps.

**Variables:**

* Angles
    * Bus 1 time 1
    * Bus 1 time 2
    * Bus 2 time 1
    * Bus 2 time 2
* Deviations
    * Bus 1 time 1
    * Bus 1 time 2
    * Bus 2 time 1
    * Bus 2 time 2

**Data input:**

* Generation
    * `GEN_EST` time 1
    * `GEN_EST` time 2
* Demand
    * Demand time 1
    * Demand time 2
* Parameters
    * Admittance matrix
    * $\Delta T_{lim}$

## Low-dimensional manifestation

$$\begin{align}
\min x^\top x &\\
s.t.&\\
\begin{bmatrix}Y & 0\\ 0 & Y\end{bmatrix}\theta - x &= G - D \\
\theta^\top Q\theta &= c
\end{align}$$

where

* $x = \begin{bmatrix}dev_1^{(1)} & dev_2^{(1)} & dev_1^{(2)} & dev_2^{(2)}\end{bmatrix}$ is the vector of deviations from wind forecast.

* $Y$ is the admittance matrix.

* $\theta = \begin{bmatrix} \theta_1^{(1)} & \theta_2^{(1)} & \theta_1^{(2)} & \theta_2^{(2)}\end{bmatrix}^\top$ is the vector of angles.

* $c$ is a scaled and derated version of the line temperature constraint:
$$c = \left(\Delta T_{12} - \delta_{12}\begin{bmatrix} \tau & 1\end{bmatrix}\begin{bmatrix}\Delta d_{12}^{(1)} \\ \Delta d_{12}^{(2)}\end{bmatrix}\right)\left(\frac{x_{12}^2}{\rho_{12}r_{12}}\right)$$

* $Q$ is symmetrix and block diagonal as shown:
$$\begin{align}
(\tau)\left[\left(\theta_1^{(1)} - \theta_2^{(1)}\right)^2\right] + (1)\left[\left(\theta_1^{(2)} - \theta_2^{(2)}\right)^2\right]
&= \begin{bmatrix} \theta_1^{(1)} & \theta_2^{(1)} & \theta_1^{(2)} & \theta_2^{(2)}\end{bmatrix} \underbrace{\begin{bmatrix} \tau & - \tau & 0 & 0 \\ -\tau & \tau & 0 & 0 \\ 0 & 0 & 1 & -1 \\ 0 & 0 & -1 & 1 \end{bmatrix}}_Q \begin{bmatrix} \theta_1^{(1)} \\ \theta_2^{(1)} \\ \theta_1^{(2)} \\ \theta_2^{(2)}\end{bmatrix}
\end{align}$$

The form of the problem suggests that we eliminate bus angles completely.

## General Problem

$$
\begin{align}
(P)& & \min x^\top x &\\
&& s.t.&\\
&&\text{diag }(Y,Y,\cdots,Y)\theta - x &= G - D \\
&&\theta^\top Q\theta &= c
\end{align}
$$

* $x\in \mathbb{R}^{mT\times1}$, where $m$ is the number of nodes in the network and $T$ is the number of time steps, is the vector of decision variables:
$$
\begin{align}
x = \begin{bmatrix} dev_1^{(1)} & dev_2^{(1)} & \cdots & dev_n^{(1)} &
dev_1^{(2)} & dev_2^{(2)} & \cdots & dev_n^{(T)} \end{bmatrix}^\top
\end{align}
$$
These variables represent deviation from wind forecast at each wind farm. Of course, the deviation at a node without a wind farm is identically zero.

* $Y\in\mathbb{R}^{m\times m}$ is the admittance matrix.

* $\theta\in\mathbb{R}^{mT\times 1}$, where $m$ is the number of nodes in the network, is the vector of angle variables, which are internal. (If we assume bus $i$ is the system slack bus, then $T$ of the angle variables --those corresponding to node $i$-- are identically zero, and we remove row $i$ and column $i$ from $Y$.)
$$
\begin{align}
\theta = \begin{bmatrix} \theta_1^{(1)} & \theta_2^{(1)} & \cdots & \theta_{m-1}^{(1)} &
\theta_1^{(2)} & \theta_2^{(2)} & \cdots & \theta_{m-1}^{(T)} \end{bmatrix}^\top
\end{align}
$$

* $G$ and $D$ are $mT\times 1$ vectors of conventional generation and demand, respectively, at each node.

* $c_{ik}$ is a scaled and derated version of the temperature constraint for line $i\to k$:
$$
\begin{align}
c_{ik} &= \left(\Delta T_{ik}^{lim} - \delta_{ik}\begin{bmatrix} \tau^{T-1} & \tau^{T-2} & \cdots & \tau & 1\end{bmatrix}\begin{bmatrix}\Delta d_{ik}^{(1)} \\ \Delta d_{ik}^{(2)} \\ \vdots \\ \Delta d_{ik}^{(T)}\end{bmatrix}\right)\left(\frac{x_{ik}^2}{\rho_{ik}r_{ik}}\right)
\end{align}
$$

* $Q\in\mathbb{R}^{(m-1)T\times (m-1)T}$ is a symmetric block-diagonal matrix that may be expressed as:
$$
\begin{align}
Q &= \text{diag}\left( \tau^{T-1} q_{ik}q_{ik}^\top, \tau^{T-2} q_{ik}q_{ik}^\top,\cdots, q_{ik}q_{ik}^\top \right),\quad q_{ik} = e_i - e_k~,
\end{align}
$$
where $e_i\in\mathbb{R}^{(m-1)T\times 1}$ is a vector with 1 in the $i$-th position and 0 everywhere else.

Note: If we had $\theta^\top Q\theta - c \leq 0$ instead of $\theta^\top Q\theta = 0$, we could consider $(P)$ to be a convex problem and the KKT conditions to be sufficient for optimality.

### Question: Can we eliminate the deviation variables, expressing everything in terms of $\theta$?

\begin{align}
&(P)&  \min \left( \text{diag }(Y,Y,\cdots,Y)\theta - G + D \right)^\top \left( \text{diag }(Y,Y,\cdots,Y)\theta - G + D \right) &\\
&& s.t.&\\
&&\theta^\top Q\theta &= c
\end{align}